In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [7]:
!pip install pytesseract

  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
Using cached pytesseract-0.3.13-py3-none-any.whl (14 kB)


In [8]:
import cv2
import pytesseract
import numpy as np
from PIL import Image


In [9]:

# Ensure Tesseract is in your PATH, or provide the direct path here
pytesseract.pytesseract.tesseract_cmd = r'/bin/tesseract'



In [10]:
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames


In [11]:

def extract_probability_via_ocr(frame):
    # Pre-processing for better OCR results (optional, adjust as needed)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Assuming probabilities are displayed in a consistent, readable font
    probability_text = pytesseract.image_to_string(thresh, config='--psm 11')

    # Basic parsing to extract probability values (adjust based on actual text output)
    import re
    probabilities = [float(match) for match in re.findall(r"(\d\.\d+)", probability_text)]

    return probabilities


In [ ]:

# Load Video
video_path = '/content/drive/MyDrive/punches/yolov10/runs/detect/predict2/test2.avi'
frames = extract_frames(video_path)


In [ ]:

# Example: Extract and Print Probabilities for the First Few Frames
for i, frame in enumerate(frames[:10]):
    probabilities = extract_probability_via_ocr(frame)
    print(f"Frame {i+1} Probabilities: {probabilities}")

In [ ]:
def analyze_consecutive_frames(frames, threshold=0.5, consecutive_frames_required=3):
    count_instances = 0
    in_sequence = False
    sequence_length = 0

    for frame in frames:
        probabilities = extract_probability_via_ocr(frame)
        frame_meets_threshold = any(probability >= threshold for probability in probabilities)

        if frame_meets_threshold:
            if not in_sequence:
                in_sequence = True
                sequence_length = 1
            else:
                sequence_length += 1
        else:
            if in_sequence and sequence_length >= consecutive_frames_required:
                count_instances += 1
            in_sequence = False
            sequence_length = 0

    # Check the last sequence
    if in_sequence and sequence_length >= consecutive_frames_required:
        count_instances += 1

    return count_instances



In [ ]:
# Execute Analysis
instances_count = analyze_consecutive_frames(frames)
print(f"Instances with 3+ consecutive frames above 50% probability: {instances_count}")